In [1]:
#Created by: Georgina Hoagland
#Created: Aug 29, 2016
#Last Modified: October 3, 2016

#To check: Day counters and calendar conventions
#To add: Units

from QuantLib import *

#setup global eval date 
calc_date = Date(29, 9, 2016)
Settings.instance().evaluationDate = calc_date

#Calendar, rules, conventions

calendars = [Argentina, Australia, BespokeCalendar, Brazil, Canada, China, \
             CzechRepublic, Denmark, Finland, Germany, HongKong, Hungary, Iceland, \
             India, Indonesia, Israel, Italy, Japan, JointCalendar, Mexico, NewZealand,\
             Norway, NullCalendar, Poland, Romania, Russia, SaudiArabia, Singapore, \
             Slovakia, SouthAfrica, SouthKorea, Sweden, Switzerland, Taiwan, TARGET, \
             Turkey, Ukraine, UnitedKingdom, UnitedStates, WeekendsOnly]

bus_day_conventions = [Following, ModifiedFollowing, Preceding, ModifiedPreceding, \
                       Unadjusted,HalfMonthModifiedFollowing]

gen_rules = [DateGeneration.Backward, DateGeneration.Forward, DateGeneration.Zero, \
             DateGeneration.ThirdWednesday] 



In [2]:
#Swap info

settlementDays = 2
    
fixedLegNotional = 100 #dollars
fixedLegTenor = Period(6, Months)
fixedLegFrequency = Semiannual
fixedLegAdjustment = ModifiedFollowing
fixedLegDayCounter = Thirty360()

floatingLegSpread = 0.0 #Libor rate - Treasury security of same maturity
floatingLegTenor = Period(6, Months) #Libor 6M
floatingLegFrequency = Semiannual
#floatingLegAdjustment = ModifiedFollowing
#floatingLegDayCounter = Thirty360()
#When changed to Actual360 to correspond with Libor conventions,\
#NPV for maturities that are traded (i.e. 1, 7, 10, 20, 25Y and not 11, 17, 24Y)\
#moved to the e-12 - e-15 range?

# term structure handles
discountTermStructure = RelinkableYieldTermStructureHandle()
forwardTermStructure = RelinkableYieldTermStructureHandle() 
# RelinkableHandle<YieldTermStructure>

#need to change to be able to change currency?
liborIndex = USDLibor(floatingLegTenor, forwardTermStructure)

floatingLegDayCounter = liborIndex.dayCounter()
floatingLegAdjustment = liborIndex.businessDayConvention()


#use index in from lists to choose calendar, convention, etc.
eval_date = calc_date
cal = calendars[-6]() #TARGET

gen_rule = gen_rules[0] # Backward 
end_of_month = True

settlementDate = cal.advance(eval_date, settlementDays, Days, floatingLegAdjustment)

#Unadjusted calendar for termination purposes
termination_cal = calendars[-18]() #NullCalendar
termination_convention = bus_day_conventions[4] #Unadjusted

In [3]:
#Inputs

ois = {   (1,Days): 0.004,
          (1,Weeks): 0.004,
          (2,Weeks): 0.004,
          (3,Weeks): 0.00417,
          (1,Months): 0.004175,
          (2,Months): 0.00429,
          (3,Months): 0.004366,
          (4,Months): 0.004515,
          (5,Months): 0.00472,
          (6,Months): 0.00485,
          (9,Months): 0.00518,
          (12,Months): 0.005445,
          (18,Months): 0.005801,
          (2,Years): 0.006070,
          (3,Years): 0.006560,
          (4,Years): 0.007010,
          (5,Years): 0.007490,
          (7,Years): 0.012512,
          (10,Years): 0.013925,
          (12,Years): 0.014701,
          (15,Years): 0.015529,
          (20,Years): 0.016385,
          (25,Years): 0.016780,
          (30,Years): 0.016989,
          (40,Years): 0.016995,
          (50,Years): 0.017085}

#convert to ql Quote objects
for n,unit in ois.keys():
    ois[(n,unit)] = SimpleQuote(ois[(n,unit)])


swaps = { (1,Years): 0.00900,
          (2,Years): 0.00975,    
          (3,Years): 0.01040,
          (4,Years): 0.01099,
          (5,Years): 0.01160,
          (6,Years): 0.01224,
          (7,Years): 0.01284,
          (8,Years): 0.01343,
          (9,Years): 0.01390,
          (10,Years): 0.01438,
          (15,Years): 0.01610,
          (20,Years): 0.01707,
          (25,Years): 0.01752,
          (30,Years): 0.01778} # 1 Year = 90bps, etc. 


#convert swap quotes from floats to Quote objects
for (n,unit) in swaps.keys():
    swaps[(n,unit)] = SimpleQuote(swaps[(n,unit)])


In [4]:
#Schedule

#creates dicts with keys of year term and values of termination date/payment schedule 
termination_date = {}
for i in range(1, 31):
    termination_date[i] = termination_cal.advance(settlementDate, Period(i, Years), \
                                                  termination_convention)
fixedSchedule = {}
for i in range(1, 31):
    fixedSchedule[i] = Schedule(settlementDate, termination_date[i], fixedLegTenor, cal, \
                                   fixedLegAdjustment, fixedLegAdjustment, gen_rule, \
                                   end_of_month)
floatingSchedule = {}
for i in range(1, 31):
    floatingSchedule[i] = Schedule(settlementDate, termination_date[i], floatingLegTenor, \
                                   cal, floatingLegAdjustment, floatingLegAdjustment, \
                                   gen_rule, end_of_month)

In [6]:
#Bootstrapping 

swapHelpers =  [SwapRateHelper(QuoteHandle(swaps[(n,unit)]), Period(n,unit), cal,\
                                     fixedLegFrequency, fixedLegAdjustment, \
                                     fixedLegDayCounter, liborIndex)\
                      for (n, unit) in swaps.keys() ]

oisHelpers = [OISRateHelper(settlementDays, Period (n, unit), QuoteHandle(ois[(n, unit)]), \
                            liborIndex)\
                     for (n, unit) in ois.keys()]

#PiecewiseYieldCurve does not work in python
#currently using PiecewiseFlatForward - corresponding to \
#PiecewiseYieldCurve<ForwardRate,BackwardFlat> in C++
#check day counter - using fixedLegDayCounter

piecewiseYieldCurve = PiecewiseFlatForward(settlementDays, cal, swapHelpers, \
                                           fixedLegDayCounter)
oisCurve = PiecewiseFlatForward(settlementDays, cal, oisHelpers, fixedLegDayCounter)

NotImplementedError: Wrong number or type of arguments for overloaded function 'new_OISRateHelper'.
  Possible C/C++ prototypes are:
    OISRateHelperPtr::OISRateHelperPtr(Natural,Period const &,Handle< Quote > const &,OvernightIndexPtr const &,Handle< YieldTermStructure > const &)
    OISRateHelperPtr::OISRateHelperPtr(Natural,Period const &,Handle< Quote > const &,OvernightIndexPtr const &)


In [136]:
#Swap additions - linear
#uses averages to assign rates to non-existent swap maturities, i.e. 11Y, 12Y, 23Y
swapsAdditionsLinear = {}
for i in range(11, 31):
    if i % 5 != 0:
        swapsAdditionsLinear [(i, Years)] = \
                        ((swaps[((i//5 + 1) * 5, Years)].value() \
                          - swaps[((i//5)*5, Years)].value()) \
                          *(i % 5)/5) + swaps[((i//5)*5, Years)].value()
for (i, unit) in swapsAdditionsLinear.keys():
    swapsAdditionsLinear[(i, unit)] = SimpleQuote(\
                                            swapsAdditionsLinear[(i, unit)])
            
#both the given swap rates and intermediate values for non-existant maturities        
swapsAll = {}
for (i, unit) in swaps.keys():
    swapsAll[(i, unit)] = swaps[(i, unit)]
for (i, unit) in swapsAdditionsLinear.keys():
    swapsAll[(i, unit)] = swapsAdditionsLinear[(i, unit)]

In [139]:
#Swap Pricing

swapTypes = [VanillaSwap.Payer, VanillaSwap.Receiver]
swapType = swapTypes[0]

#discountTermStructure.linkTo(piecewiseYieldCurve)
#forwardTermStructure.linkTo(piecewiseYieldCurve)
discountTermStructure.linkTo(oisCurve)
forwardTermStructure.linkTo(oisCurve)
swapEngine = DiscountingSwapEngine(discountTermStructure)

    
vanillaSwaps = {}
for i in range(1, 31):
    vanillaSwaps[(i, Years)] = VanillaSwap(swapType, fixedLegNotional, fixedSchedule[i],\
                        swapsAll[(i, Years)].value(), fixedLegDayCounter, \
                        floatingSchedule[i], liborIndex, floatingLegSpread,\
                        floatingLegDayCounter)
    
    vanillaSwaps[(i, Years)].setPricingEngine(swapEngine)
    
swapsNPV = {}
for i in range(1, 31):
    swapsNPV[(i, Years)] = vanillaSwaps[(i, Years)].NPV()
    print i, swapsNPV[(i, Years)]

1 -0.347397179064
2 -0.715941220168
3 -1.11596937715
4 -1.53582961102
5 -1.97197635369
6 -0.989367109786
7 -0.128339508396
8 -0.142653692106
9 -0.166516454578
10 -0.286679683581
11 -0.226501910187
12 -0.231697217471
13 -0.276907667479
14 -0.384661309699
15 -0.553221481746
16 -0.55185259551
17 -0.586893636475
18 -0.655155536251
19 -0.756439795318
20 -0.889830326108
21 -0.916783511526
22 -0.959808064042
23 -1.01733265657
24 -1.08867526708
25 -1.17380215305
26 -1.21747502214
27 -1.26856302933
28 -1.32792939844
29 -1.39431218444
30 -1.46787325335


In [140]:
#Swaption Volatility

#using constant value of 40%
volatilityFlat = 0.40

volatilityStructure = ConstantSwaptionVolatility(settlementDays, cal,\
                                                     fixedLegAdjustment, volatilityFlat, \
                                                     fixedLegDayCounter)
# default: VolatilityType type=ShiftedLognormal, const Real shift=0.0

volatilityStructureHandle = SwaptionVolatilityStructureHandle(volatilityStructure)


In [141]:
#Swaption Inputs

swaptionMaturity = Period(3, Months)
swaptionExpiry = cal.advance(settlementDate, swaptionMaturity, floatingLegAdjustment)
settlementType = Settlement.Physical

#European
swaptionExercise = EuropeanExercise(swaptionExpiry)

In [142]:
#Swaption Object

swaptionPricingEngine = BlackSwaptionEngine(forwardTermStructure, volatilityStructureHandle)

swaptions = {}
for i in range(1, 31):
    swaptions [(i, Years)] = Swaption(vanillaSwaps[(i, Years)], swaptionExercise, \
                                      settlementType)
    
    swaptions[(i, Years)].setPricingEngine(swaptionPricingEngine)

In [143]:
#Swaption Pricing

swaptionsNPV = {}
for maturity in swaptions.keys():
    swaptionsNPV[maturity] = swaptions[maturity].NPV()
    
for i in range(1, 31):
    print i, swaptionsNPV[(i, Years)]


1 0.000325303719317
2 0.00109244019402
3 0.00211805840285
4 0.00354041905414
5 0.00553264778379
6 0.175939102676
7 0.625195574045
8 0.745902263811
9 0.860269035704
10 0.932199809793
11 1.08784084815
12 1.21342751064
13 1.32145889727
14 1.40049741091
15 1.45264729278
16 1.57431001786
17 1.67798983017
18 1.76601706111
19 1.83841051674
20 1.89598400312
21 1.99094993359
22 2.07663615628
23 2.15459983915
24 2.22546628871
25 2.28907051549
26 2.36528111652
27 2.43735676484
28 2.50348753863
29 2.5657391514
30 2.62360039193


In [144]:
#Implied Volatility

swaptionHelpers = {}
impliedVols = {}
for i in range(1, 31):
    swaptionHelpers[(i, Years)] = SwaptionHelper(swaptionMaturity, Period(i, Years), \
                                  QuoteHandle(SimpleQuote(volatilityFlat)), \
                                  liborIndex, fixedLegTenor, fixedLegDayCounter, \
                                  floatingLegDayCounter, forwardTermStructure)

    swaptionHelpers[(i, Years)].setPricingEngine(swaptionPricingEngine)

#    impliedVols[(i, unit)] = swaptionHelpers[(i, unit)].impliedVolatility(\
#                                                            swaptionsNPV[(i, unit)], \
#                                                            1e-7, 50, 0.0, 10.0)\
                             
#input values my be causing runtime errors in the implied vol value                